dnn-hy 实验二 by 孟


基础航班延误模型，2分类任务，之前下采样【-500，30】，【30，-】，现在下采样【-500，120】，【120，-】相比于之前的二分类任务，准确率有明显提升。

之前 66.5%   现在 70.7%


In [1]:
import keras

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
from __future__ import print_function

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn import preprocessing
from sklearn.utils import shuffle 
from sklearn.model_selection import KFold, train_test_split

from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

from keras import initializers
from keras.callbacks import Callback
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop

%matplotlib inline

In [3]:
data = pd.read_feather('/home/ubuntu/data/flight_data_7_18.feather')

In [4]:
data['feat1'] = data['delay_cat'].map(lambda x: 1 if x == '[-500,30)' or x == '[30,60)' or x == '[60,90)' or x == '[90,120)' else 0)
data['feat2'] = data['feat1'].map(lambda x: 1-x)
data = data.drop(['delay_cat'], axis=1)

In [5]:
data = data.dropna()
#data = shuffle(data)
data = data.sample(frac=0.5)
#print(data.info())

In [6]:
data['feat1'].value_counts()

1    3296578
0     226324
Name: feat1, dtype: int64

In [7]:
from sklearn.utils import resample

data_majority = data[data.feat1==1]
data_minority = data[data.feat1==0]
data_majority_downsampled = resample(data_majority,
                                     replace=False,    # sample without replacement
                                     n_samples=226020,     # to match minority class
                                     random_state=1) # reproducible results
data_downsampled = pd.concat([data_majority_downsampled, data_minority])

In [8]:
data = data_downsampled

In [9]:
data['sch_time_year'] = data['sch_dep_time'].map(lambda x: x.year)
data['sch_time_month'] = data['sch_dep_time'].map(lambda x: x.month)
data['sch_time_day'] = data['sch_dep_time'].map(lambda x: x.day)
data['sch_time_hour'] = data['sch_dep_time'].map(lambda x: x.hour)

In [10]:
data['sch_time_year'] = (data['sch_time_year'].map(lambda x: int(str(x)[2:])))
month_num = {1:6, 2:2, 3:2, 4:5, 5:0, 6:3, 7:5, 8:1, 9:4, 10:6, 11:2, 12:4}
data['year_num'] = data['sch_time_year'].map(lambda x: (int(x / 4) + x) % 7)
data['month_num']=  data['sch_time_month'].map(lambda x: month_num[x])
data['day_num'] = data['sch_time_day']
data['sch_time_day'] = data.apply(lambda x: (x['year_num'] + x['month_num'] + x['day_num'])%7, axis=1)
time_drop = ['year_num', 'month_num', 'day_num']
data = data.drop(time_drop,axis=1)

In [11]:
data['sch_time_hour'] = data['sch_time_hour'].map(lambda x: 24 if x==0 else x)
for i in range(8):  #每3个小时一个区间
    data['sch_time_hour'] = data['sch_time_hour'].map(lambda x: 0+i if x>0+3*i and x<=3+3*i else x)

In [12]:
lb = ['feat1','feat2']
label = data[lb]
data = data.drop(lb, axis=1)
data = data.drop('sch_dep_time', axis=1)

In [13]:
data1 = data.iloc[:,3:24]

data2 = data.iloc[:,0:3]
data3 = data.iloc[:,24:]
data2 = pd.concat([data2, data3], axis=1)
#print(data1.columns, data2.columns)
data1 = (data1 - data1.mean()) / data1.std()

In [14]:
data = pd.concat([data1, data2], axis=1)

In [15]:
data_train = data
label_train = label

In [16]:
one_hot = ['sch_time_year','sch_time_month','sch_time_day','sch_time_hour','airline','origin_airport','dest_airport']
data_train = pd.get_dummies(data_train, columns=one_hot)

In [17]:
clf = ExtraTreesClassifier()
clf = clf.fit(data_train, label_train)

In [18]:
importance = clf.feature_importances_ 
for i in range(len(importance)):
    if importance[i]>=1e-1 or importance[i]<=1e-5:
        importance[i] = 0
feature_importance_index = np.where(importance!=0)
#feature_importance_index
#len(feature_importance_index)

In [19]:
feature_importance_index = np.squeeze(np.array(feature_importance_index))

In [20]:
data_train, data_test, label_train, label_test = train_test_split(data_train, label_train, test_size=0.2, random_state=2)

In [21]:
from keras.optimizers import SGD

input_size = len(data_train.columns)

inputs = Input(shape=(input_size,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
#x = Dropout(0.5)(x)
#x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
#x = Dense(128, activation='relu')(x)
#x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(2, activation='softmax', name='ou')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model.fit(x=data_train,
          y=label_train,
          epochs=100,
          batch_size=256,
          #verbose=2,
          validation_split=0.2
          )

Train on 289500 samples, validate on 72375 samples
Epoch 1/100
289500/289500 [==============================] - 8s 28us/step - loss: 0.6634 - acc: 0.5991 - val_loss: 0.6332 - val_acc: 0.6460
Epoch 2/100
289500/289500 [==============================] - 7s 25us/step - loss: 0.6364 - acc: 0.6406 - val_loss: 0.6197 - val_acc: 0.6578
Epoch 3/100
289500/289500 [==============================] - 7s 25us/step - loss: 0.6252 - acc: 0.6528 - val_loss: 0.6139 - val_acc: 0.6629
Epoch 4/100
289500/289500 [==============================] - 7s 25us/step - loss: 0.6196 - acc: 0.6584 - val_loss: 0.6101 - val_acc: 0.6663
Epoch 5/100
289500/289500 [==============================] - 7s 24us/step - loss: 0.6155 - acc: 0.6619 - val_loss: 0.6070 - val_acc: 0.6690
Epoch 6/100
289500/289500 [==============================] - 7s 24us/step - loss: 0.6117 - acc: 0.6658 - val_loss: 0.6040 - val_acc: 0.6722
Epoch 7/100
289500/289500 [==============================] - 7s 25us/step - loss: 0.6088 - acc: 0.6688 - val_

Epoch 59/100
289500/289500 [==============================] - 8s 29us/step - loss: 0.5560 - acc: 0.7188 - val_loss: 0.5704 - val_acc: 0.7046
Epoch 60/100
289500/289500 [==============================] - 9s 30us/step - loss: 0.5549 - acc: 0.7189 - val_loss: 0.5700 - val_acc: 0.7055
Epoch 61/100
289500/289500 [==============================] - 9s 30us/step - loss: 0.5549 - acc: 0.7198 - val_loss: 0.5700 - val_acc: 0.7056
Epoch 62/100
289500/289500 [==============================] - 8s 29us/step - loss: 0.5544 - acc: 0.7204 - val_loss: 0.5701 - val_acc: 0.7048
Epoch 63/100
289500/289500 [==============================] - 8s 29us/step - loss: 0.5540 - acc: 0.7200 - val_loss: 0.5703 - val_acc: 0.7063
Epoch 64/100
289500/289500 [==============================] - 9s 30us/step - loss: 0.5534 - acc: 0.7202 - val_loss: 0.5701 - val_acc: 0.7055
Epoch 65/100
289500/289500 [==============================] - 8s 29us/step - loss: 0.5537 - acc: 0.7202 - val_loss: 0.5701 - val_acc: 0.7053
Epoch 66/100


In [23]:
score = model.evaluate(x=data_test,
                       y=label_test,
                       batch_size=len(data_test))

print('Test score:', score[0])
print('Test accuracy:', score[1])

90469/90469 [==============================] - 0s 4us/step
Test score: 0.5667464733123779
Test accuracy: 0.7079331278800964
